In [20]:
import scipy.stats as stats
import math
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pandas as pd
from statsmodels.stats.weightstats import ztest as ztest


def display_result(test_stat, crit_val, p_val=0, alpha=0):
    print("="*40)
    if test_stat >= crit_val or p_val < alpha:
        print("H0 is REJECTED\nH1 is ACCEPTED\n\nThere is a significant difference")
    elif test_stat < crit_val or p_val >= alpha:
        print("H0 is ACCEPTED\n\nThere is NO significant difference")
    print("="*40)


def z_test_1(n, samp_mean, mu, stdev, alpha):
    test_stat = (samp_mean - mu) / (stdev / math.sqrt(n))
    crit_val_1 = abs(stats.norm.ppf(1-alpha))
    crit_val_2 = abs(stats.norm.ppf(1-alpha/2))
    p_val_1 = stats.norm.sf(abs(test_stat))
    p_val_2 = stats.norm.sf(abs(test_stat) / 2)
    
    return test_stat, crit_val_1, crit_val_2, p_val_1, p_val_2


def z_test_arr(arr, x, stdev=None):
    if stdev == None: stdev=statistics.stdev(arr)
    z = (np.mean(arr) - x) / (stdev / math.sqrt(len(arr)))
    return 1-stats.norm.cdf(z)


def z_test_2samp(a, b, alpha):
    mean_a = np.mean(a)
    mean_b = np.mean(b)

    alpha = 0.05

    crit_val_1 = stats.norm.ppf(1-alpha)
    crit_val_2 = stats.norm.ppf(1 - alpha/2)
    variance_a = np.var(a)
    variance_b = np.var(b)

    test_stat, p_val = ztest(a, b, value=abs(mean_a-mean_b))

    return mean_a, mean_b, crit_val_1, crit_val_2, variance_a, variance_b, test_stat, p_val


def get_chi_E(O, total_er, total_ec, total):
    lo = len(O)
    E = [[] for _ in range(lo)]
    for i in range(lo):
        lr = len(O[i])
        for j in range(lr):
            E[i].append((total_er[i] * total_ec[j]) / total)
    return E


def parse_samp(*args):
    samples = []
    total_arr = [sum(r) for r in args]
    nsamp = len(args)
    for r in args:
        for n in r:
            samples.append(n)
    samp_len = len(samples)
    return samples, total_arr, samp_len, nsamp


def display_anova_summ(*args):
    print("===== SUMMARY =====")
    print("Group\t    Sum\t    Average\t    Variance")
    for i in range(len(args)):
        print(f"G{i}\t{sum(args[i]):.6f}\t{np.mean(args[i]):.6f}\t{statistics.variance(args[i]):.6f}")
    print("===================")

### $Z$-test

In [2]:
n = 25
samp_mean = 372.5
mu = 368
stdev = 15
alpha = 0.05

test_stat, crit_val_1, crit_val_2, p_val_1, p_val_2 = z_test_1(n, samp_mean, mu, stdev, alpha)

print(f"""
Z = {test_stat}

CRITICAL VALUE (one-tail) = {crit_val_1:.6f}
P-VALUE (one-tail) = {p_val_1:.6f}

CRITICAL VALUE (two-tail) = {crit_val_2:.6f}
P-VALUE (two-tail) = {p_val_2:.6f}
""")

display_result(test_stat, crit_val_1, p_val_1, alpha)


Z = 1.5

CRITICAL VALUE (one-tail) = 1.644854
P-VALUE (one-tail) = 0.066807

CRITICAL VALUE (two-tail) = 1.959964
P-VALUE (two-tail) = 0.226627

H0 is ACCEPTED

There is NO significant difference


### $Z$-test with array

In [3]:
a = [
    4,5,6,7,9,12,15,16
]

x_norm = 4

prob_value = z_test_arr(a, x_norm)

print(f"""
One-tailed prob value = {prob_value}
Two-tailed prob value = {2 * prob_value}
""")


One-tailed prob value = 0.0006084831041472949
Two-tailed prob value = 0.0012169662082945898



### $Z$-test with 2 samples

In [4]:
a = [
    14,16,20,10,21,14,19,8,18,15,9,15,18,7,17,19,13,18,14,12,15,29,21,17,13,12,13,14
    
    #93,68,82,51,87,82,72,71,92,73
]
b = [
    11,9,5,17,11,15,9,19,3,12,8,20,16,18,10,21,14,13,18,14,18,30,25,27,23,18,15,12
    
    #79,66,73,72,80,74,95,93,99,97
]

alpha = 0.05

mean_a, mean_b, crit_val_1, crit_val_2, variance_a, variance_b, test_stat, p_val = z_test_2samp(a, b, alpha)


print(f"""
A mean = {mean_a}
A Variance = {variance_a}

B mean = {mean_b}
B Variance = {variance_b}

Mean diff = {mean_a - mean_b}

Z = {test_stat}

CRITICAL VALUE (one-tail) = {crit_val_1}
P-VALUE (one-tail) = {p_val/2}

CRITICAL VALUE (two-tail) = {crit_val_2}
P-VALUE (two-tail) = {p_val}
""")

display_result(test_stat, crit_val_2, p_val, alpha)


A mean = 15.392857142857142
A Variance = 20.167091836734695

B mean = 15.392857142857142
B Variance = 39.02423469387755

Mean diff = 0.0

Z = 0.0

CRITICAL VALUE (one-tail) = 1.6448536269514722
P-VALUE (one-tail) = 0.5

CRITICAL VALUE (two-tail) = 1.959963984540054
P-VALUE (two-tail) = 1.0

H0 is ACCEPTED

There is NO significant difference


### Student $t$-test

In [5]:
mean_pop = 800 # sample
mean_norm = 778 # normal
n = 30
stdev = 40
alpha = 0.05
type2_prob = 1 - alpha
df = n - 1

test_stat = (abs(mean_pop - mean_norm) * math.sqrt(n)) / stdev
crit_val = stats.t.ppf(1-alpha, df)

print(f"""
t = {test_stat}
CRITICAL VALUE = {crit_val}
""")
display_result(test_stat, crit_val)


t = 3.012474066278414
CRITICAL VALUE = 1.6991270265334972

H0 is REJECTED
H1 is ACCEPTED

There is a significant difference


### $t$-test for correlated samples

In [6]:
before = [
    #4.2,4.7,6.6,7.0,6.7,4.5,5.7,6.0,7.4,4.9,6.1,5.2
    
    #45,45,45,50,55,80
    
    90.563, 94.816, 109.56, 90.222, 97.598, 91.167, 96.65, 97.616, 88.845, 90.817, 89.294, 115.83, 121.29, 87.872, 93.793
]
after = [
    #4.1,4.9,6.2,6.9,5.8,4.4,5.7,5.8,6.9,4.7,6.0,4.9
    
    #62,55,55,65,68,70
    
    110.642, 101.588, 120.607, 83.2217, 109.272, 115.806, 99.8958, 117.94, 106.052, 82.8229, 116.639, 128.61,
    119.665, 108.383, 96.3738
]

tail = "2"

alpha = 0.05

n1 = len(before)
n2 = len(after)
df = n1 - 1

before_mean = np.mean(before)    #barX_1
after_mean = np.mean(after)      # sum(after) / n2      #barX_2

stdev_before = np.std(before) #sigma1
stdev_after = np.std(after)   #sigma2

variance_before = statistics.variance(before)
variance_after = statistics.variance(after)

crit_val_1 = abs(stats.t.ppf(alpha, df))     # T_tab
crit_val_2 = abs(stats.t.ppf(1-alpha/2, df))

t, p_val = stats.ttest_rel(before, after)

print(f"""
MEAN (before) = {before_mean}
MEAN (after) = {after_mean}

STDEV_1 = {stdev_before}
STDEV_2 = {stdev_after}

VARIANCE_1 = {variance_before}
VARIANCE_2 = {variance_after}

t = {t}

CRITICAL VALUE (one-tail) = {crit_val_1}
P-VALUE (one-tail) = {p_val/2}

CRITICAL VALUE (two-tail) = {crit_val_2}
P-VALUE (two-tail) = {p_val}
""")

if tail == "1":
    crit_val = crit_val_1
    p_val = p_val/2
elif tail == "2":
    crit_val = crit_val_2

display_result(t, crit_val, p_val, alpha)


MEAN (before) = 97.0622
MEAN (after) = 107.83454666666667

STDEV_1 = 9.956637010557332
STDEV_2 = 12.796067347163955

VARIANCE_1 = 106.21566488571432
VARIANCE_2 = 175.43500666409528

t = -3.734378703414312

CRITICAL VALUE (one-tail) = 1.7613101357748564
P-VALUE (one-tail) = 0.0011104774400844042

CRITICAL VALUE (two-tail) = 2.1447866879169273
P-VALUE (two-tail) = 0.0022209548801688083

H0 is REJECTED
H1 is ACCEPTED

There is a significant difference


### $t$-test for 2 independent samples

In [7]:
a = [
    20.7, 27.6, 24.5, 34.5, 41.4, 40.0, 17.7, 25.8, 36.7, 19.8
]
b = [
    13.8, 32.2, 29.6, 21.2, 27.3, 30.1, 26.5, 15.5, 23.7, 35.0
]

tail = "1"

alpha = 0.05
n = 10
df = 10 + 10 - 2

mean_a = np.mean(a)
mean_b = np.mean(b)

variance_a = statistics.variance(a)
variance_b = statistics.variance(b)
pooled_variance = statistics.variance(a+b)

t, p_val = stats.ttest_ind(a, b, equal_var=True)

crit_val_1 = abs(stats.t.ppf(alpha, df))
crit_val_2 = abs(stats.t.ppf(1-alpha/2, df))

print(f"""
A Mean = {mean_a}
A Variance = {variance_a}
A length = {len(a)}

B Mean = {mean_b}
B Variance = {variance_b}
B length = {len(b)}

Pooled Variance = {pooled_variance}

t = {t}

CRITICAL VALUE (one-tail) = {crit_val_1}
P-VALUE (one-tail) = {p_val/2}

CRITICAL VALUE (two-tail) = {crit_val_2}
P-VALUE (two-tail) = {p_val}
""")

if tail == "1":
    crit_val = crit_val_1
    p_val = p_val/2
elif tail == "2":
    crit_val = crit_val_2

display_result(t, crit_val, p_val, alpha)


A Mean = 28.869999999999997
A Variance = 75.31122222222223
A length = 10

B Mean = 25.49
B Variance = 48.44100000000001
B length = 10

Pooled Variance = 61.625894736842106

t = 0.9608159348933784

CRITICAL VALUE (one-tail) = 1.734063606617536
P-VALUE (one-tail) = 0.17468740800737215

CRITICAL VALUE (two-tail) = 2.10092204024096
P-VALUE (two-tail) = 0.3493748160147443

H0 is ACCEPTED

There is NO significant difference


### $\chi^2$ test

In [8]:
O = [
    28,36,36,30,27,23
    #29,24,22,19,20,23,18,20,21,19,23,20
    #2,1,3,4,5,6,7,9,10,12,5,1
]
E = [
    30,30,30,30,30,30
    #21.33,21.33,21.33,21.33,21.33,21.33,21.33,21.33,21.33,21.33,21.33,21.33
    #3,1,3,4,5,6,7,9,11,13,6.5,2
]

alpha = 0.05

k = len(O)
df = k - 1

t, p_val = stats.chisquare(O, E)
crit_val = stats.chi2.ppf(1-alpha, df)

t, p_val, crit_val

print(f"""
X^2 = {t}
P-VALUE = {p_val}
CRITICAL VALUE = {crit_val}
""")

display_result(t, crit_val, p_val, alpha)


X^2 = 4.466666666666667
P-VALUE = 0.4843556738432231
CRITICAL VALUE = 11.070497693516351

H0 is ACCEPTED

There is NO significant difference


### Cross-tabulation of $\chi^2$

In [23]:
O = [
    #[71, 154, 398],
    #[4992, 2808, 2737]
    
    [21, 14],
    [3, 21]
    
    #[27, 10],
    #[16, 15]
]
nrows = len(O)
ncols = len(O[0])
alpha = 0.05

total_er = [sum(r) for r in O]
total_ec = list(pd.DataFrame(O).sum())
total = sum(total_ec)

E = get_chi_E(O, total_er, total_ec, total)

df = (nrows - 1) * (ncols - 1)

chi2 = []

for ri in range(nrows):
    r = O[ri]
    t = E[ri]
    for ci in range(ncols):
        chi2.append(((r[ci] - t[ci])**2)/t[ci])

t = sum(chi2)

p_val = 1 - stats.chi2.cdf(t, df)
crit_val = stats.chi2.ppf(1-alpha, df)

t, crit_val

print(f"""
X^2 = {t}

P-VALUE = {p_val}
CRITICAL VALUE = {crit_val}
""")

display_result(t, crit_val, p_val, alpha)


X^2 = 13.311875000000002

P-VALUE = 0.00026373050287542554
CRITICAL VALUE = 3.841458820694124

H0 is REJECTED
H1 is ACCEPTED

There is a significant difference


### One-way ANOVA

In [21]:
d1 = [11.715501, 11.981569, 8.0439292, 10.55816, 14.079463, 10.776867, 7.8602695, 11.889672, 11.9423, 13.1774]
#d1 = [17.5, 16.9, 15.8, 18.6]
d2 = [10.566155, 13.4554, 7.4188, 12.0313, 7.7766, 10.7489, 10.7270, 4.4773, 6.8038, 5.3719]
#d2 = [16.4, 19.2, 17.7, 15.4]
d3 = [10.2833, 12.1777, 10.5598, 9.6552, 8.7903, 10.8625, 10.3782, 10.1881, 11.62452, 12.305905]
#d3 = [20.3, 15.7, 17.8, 18.9]
d4 = [6.9035, 8.9901, 6.9713, 9.1604, 8.6784, 11.4438, 10.7804, 5.6668, 10.7760, 9.00876]
#d4 = [14.6, 16.7, 20.8, 18.9]
#d5 = [17.5, 19.2, 16.5, 20.5]
#d6 = [18.3, 16.2, 17.5, 20.1]

samples, total_arr, samp_len, n_samp = parse_samp(d1,d2,d3,d4)#,d5,d6)

display_anova_summ(d1,d2,d3,d4)#,d5,d6)

alpha = 0.05

dfn = n_samp - 1 # k-1
dfd = samp_len - n_samp # N - k

SSb = sum([((tr**2)/(samp_len/n_samp)) for tr in total_arr]) - (sum(total_arr)**2)/samp_len
SSt = sum([sr**2 for sr in samples]) - (sum(total_arr)**2)/samp_len
SSw = SSt - SSb

MSb = SSb / (n_samp - 1)
MSw = SSw / (samp_len - n_samp)

t, p_val = stats.f_oneway(d1,d2,d3,d4)#,d5,d6)
crit_val = stats.f.ppf(1-alpha, dfn, dfd)

print(f"""
F Ratio = {t}
P-VALUE = {p_val}
CRITICAL VALUE = {crit_val}

Total SS = {SSb + SSw}
Total df = {dfn + dfd}

============================
BETWEEN GROUPS
--------------
df = k - 1 = {dfn}
Sum squares = {SSb}
Mean squares = {MSb}

==============

WITHIN GROUPS
-------------
df = N - k = {dfd}
Sum squares = {SSw}
Mean square = {MSw}
============================
""")

display_result(t, crit_val, p_val, alpha)

===== SUMMARY =====
Group	    Sum	    Average	    Variance
G0	112.025131	11.202513	3.978910
G1	89.377155	8.937715	8.881413
G2	106.825525	10.682552	1.215342
G3	88.379460	8.837946	3.531559

F Ratio = 3.303157054461315
P-VALUE = 0.031053029855115422
CRITICAL VALUE = 2.86626555094018

Total SS = 202.08458395572006
Total df = 39

BETWEEN GROUPS
--------------
df = k - 1 = 3
Sum squares = 43.61956926375706
Mean squares = 14.539856421252352


WITHIN GROUPS
-------------
df = N - k = 36
Sum squares = 158.465014691963
Mean square = 4.401805963665639

H0 is REJECTED
H1 is ACCEPTED

There is a significant difference


### Two-wa y ANOVA